## 서울사고건수 LSTM 예시

In [27]:
#필요한 라이브러리
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [29]:
#파일 불러오기
filename = "/content/drive/MyDrive/서울사고건수(2017~2019).csv"
df = pd.read_csv(filename)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   발생일     1095 non-null   object
 1   사고건수    1095 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 17.2+ KB
None


In [30]:
df['발생일'] = pd.to_datetime(df['발생일'])
df.set_axis(df['발생일'], inplace=True)
df.head()

,발생일,사고건수
발생일,,
2017-01-01,2017-01-01,73
2017-01-02,2017-01-02,82
2017-01-03,2017-01-03,88
2017-01-04,2017-01-04,115
2017-01-05,2017-01-05,103


In [31]:
#교육 및 예측을 수행하기 전에 데이터가 어떻게 표시되는지 plotly 사용하여 시각화

import plotly
import plotly.graph_objects as go
print("Plotly Version: ",plotly.__version__)

trace = go.Scatter(
    x = df['발생일'],
    y = df['사고건수'],
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = "",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Accident"}
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()

Plotly Version:  5.5.0


In [32]:
#데이터 사전처리
#분석을 위해 처음 80%의 데이터에 대해 모델을 교육하고 나머지 20%에 대해 테스트

accident_data = df['사고건수'].values
accident_data = accident_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(accident_data))

accident_train = accident_data[:split]
accident_test = accident_data[split:]

date_train = df['발생일'][:split]
date_test = df['발생일'][split:]

print(len(accident_train))
print(len(accident_test))

876
219


- 교육을 실시하기 전에 데이터를 크게 수정해야 함
- 우리의 데이터는 여전히 시퀀스고 neural network은 supervised model로 훈련되기 때문에 데이터를 시퀀스에서 supervised data로 변환해야 함
- 기계 학습 모델의 신경망을 훈련하려면 데이터가 {<features>,<target>} 형식이어야 함
- 마찬가지로 주어진 데이터를 이 형식으로 변환해야 함 => 룩백

In [33]:
#Keras의 시계열 생성기(TimeseriesGenerator) 모듈 사용

look_back = 15


train_generator = TimeseriesGenerator(accident_train, accident_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(accident_test, accident_test, length=look_back, batch_size=1)

In [34]:
#Neural Network, 데이터가 준비 되었으니 네트워크를 만들고 교육할 수 있음
#loss='mse'를 많이 사용

from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



44/44 [==============================] - 2s 8ms/step - loss: 8430.6953
Epoch 2/25
44/44 [==============================] - 0s 9ms/step - loss: 2801.3247
Epoch 3/25
44/44 [==============================] - 1s 11ms/step - loss: 447.0783
Epoch 4/25
44/44 [==============================] - 0s 10ms/step - loss: 419.2887
Epoch 5/25
44/44 [==============================] - 0s 9ms/step - loss: 413.3482
Epoch 6/25
44/44 [==============================] - 0s 9ms/step - loss: 396.0939
Epoch 7/25
44/44 [==============================] - 0s 9ms/step - loss: 384.8022
Epoch 8/25
44/44 [==============================] - 0s 9ms/step - loss: 384.0477
Epoch 9/25
44/44 [==============================] - 0s 8ms/step - loss: 378.7038
Epoch 10/25
44/44 [==============================] - 1s 16ms/step - loss: 377.2756
Epoch 11/25
44/44 [==============================] - 1s 11ms/step - loss: 373.4358
Epoch 12/25
44/44 [==============================] - 0s 9ms/step - loss: 386.9002
Epoch 13/25
44/44 [===========

In [35]:
#예측(Prediction, 검정 데이터에 대해 모형을 검정하고 예측과 실제 값이 겹치는지 확인할 수 있음)
#Ground Truth : 우리가 정한 정답, 우리의 모델이 우리가 원하는 답으로 예측해주길 바라는 답

prediction = model.predict_generator(test_generator)

accident_train = accident_train.reshape((-1))
accident_test = accident_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = accident_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = accident_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "-",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Accident"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [ ]:
#미래예측(Forecasting)
#내일의 값을 예측하고 싶을 경우, 지난 n일(look_back)의 값을 모델에 입력하면 내일의 값을 출력으로 얻을 수 있음
#모레 값을 얻으려면 n-1일 지난 값을 내일 값 및 모레 모델 출력 값과 함께 피드 인 하면 됨
#더 긴 기간을 예측하는 것은 실현 가능하지 않으므로 한 달 동안의 사고건수를 예측

accident_data = accident_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = accident_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['발생일'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

1/1 [==============================] - 0s 54ms/step


In [ ]:
#시각화

trace1 = go.Scatter(
    x = df['발생일'].tolist(),
    y = accident_data,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = "-",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Accident"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

## 날씨 월별로 예측